# [example](https://suhas.org/function-call-ast-python/)

* https://stackoverflow.com/questions/1515357/simple-example-of-how-to-use-ast-nodevisitor
* https://suhas.org/function-call-ast-python/
* https://docs.python.org/3/library/ast.html#ast.NodeVisitor

In [33]:
import ast
import astor  # https://stackoverflow.com/questions/36022935/rewriting-code-with-ast-python
# Python itself doesn’t provide a way to turn a compiled code object into an AST, or an AST into a string of code. https://greentreesnakes.readthedocs.io/en/latest/tofrom.html

class FunctionCallVisitor(ast.NodeVisitor):
    """
    we will use this to find all callables ("Call" nodes in an ast). then, once we
    are on a Call node, we print its information. a Call node could be defined by 
    FunctionDef but also ImportFrom.
    
    if we wanted to visit other types of nodes in the ast, we would replace visit_Call
    with visit_Assign, visit_Import, etc...
    
    https://stackoverflow.com/questions/1515357/simple-example-of-how-to-use-ast-nodevisitor
    """
    def visit_Call(self, node):
        # print(ast.dump(node))
        print(node.func.id)

In [34]:
# load source into memory
with open('hello.py') as f: 
    source = f.read()

In [35]:
# convert source code into ast representation
tree = ast.parse(source)
tree.body

In [36]:
# this is the complete ast body - this is only for illustrative purposes and we wont use this
for statement in tree.body:
    print(ast.dump(statement), '\n')

Import(names=[alias(name='ast', asname=None)]) 

ImportFrom(module='hello2', names=[alias(name='c', asname=None)], level=0) 

FunctionDef(name='a', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=2)), Return(value=BinOp(left=BinOp(left=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[]), op=Add(), right=Call(func=Name(id='b', ctx=Load()), args=[], keywords=[])), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[], returns=None) 

FunctionDef(name='b', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=2), op=Add(), right=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[])))], decorator_list=[], returns=None) 

FunctionDef(name='d', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=Num(n=4))], decorator_list=[], return

In [37]:

# only functions that are actuall called are converted into ast
# func C is called twice and therefore shows up twice in the ast
FunctionCallVisitor().visit(tree)

c
b
c


In [38]:
# then, collect its source code

for i, node in enumerate(tree.body):
    if isinstance(node, ast.FunctionDef):
        if node.name in ["a", 'b', 'c', 'd']:
            print(">> found", node.name)

>> found a
>> found b
>> found d


In [9]:
"""
first, let find all of the imports
"""
for i, statement in enumerate(tree.body):
    if isinstance(statement, ast.Import):  # https://stackoverflow.com/questions/37217823/how-to-find-detect-if-a-build-in-function-is-used-in-python-ast
        print(">> at index", i, ": \n", ast.dump(statement))

>> at index 0 : 
 Import(names=[alias(name='ast', asname=None)])


In [10]:
"""
lets find an assign statement for execution.

there are many ast node types such as ast.Assign, ast.FunctionDef, ast.import...
"""
for i, statement in enumerate(tree.body):
    if isinstance(statement, ast.Assign):  # https://stackoverflow.com/questions/37217823/how-to-find-detect-if-a-build-in-function-is-used-in-python-ast
        print(">> at index", i, ": \n", ast.dump(statement))
        idx = i

>> at index 5 : 
 Assign(targets=[Name(id='y', ctx=Store())], value=Call(func=Name(id='a', ctx=Load()), args=[], keywords=[]))


In [11]:
# we want to execute y = a() which we can subset from main node
curr = tree.body[idx]
ast.dump(curr)

"Assign(targets=[Name(id='y', ctx=Store())], value=Call(func=Name(id='a', ctx=Load()), args=[], keywords=[]))"

In [12]:
"""
NodeTransformers can return nodes but NodeVisitors cant

NodeTransformers are destructive.

"The NodeTransformer will walk the AST and use the return value of the visitor methods to replace or remove the old node. If the return value of the visitor method is None, the node will be removed from its location, otherwise it is replaced with the return value. The return value may be the original node in which case no replacement takes place."
"""

class Return_Call_Nodes(ast.NodeVisitor):  
    def visit_Call(self, node):
        # print(ast.dump(node))
        print(node.func.id)

In [13]:
# # using FunctionCallVisitor, we can find its subroutines... which is func A... 
# lets fetch the subroutines name, we cant return values so we capture stdout
# https://stackoverflow.com/questions/16571150/how-to-capture-stdout-output-from-a-python-function-call
import io
from contextlib import redirect_stdout

f = io.StringIO()
with redirect_stdout(f):
    Return_Call_Nodes().visit(curr)
out = f.getvalue()
out.split()[0]

'a'

In [14]:
# return the index of tree.body that corresponds to the name
def find_idx(target):
    for i, node in enumerate(tree.body):
        # we only want funcdef statements
        if isinstance(node, ast.FunctionDef):
            if node.name == target:
                idx = i
                # we'll grab func A's source code and save it in the dictionary
                print(">> found", node.name)
                print(astor.to_source(tree.body[idx]))
    return idx

In [15]:
idx = find_idx(out.split()[0])
idx

>> found a
def a():
    x = 2
    return c() + b() + x



2

In [410]:
# and then find func a's subroutines
curr = tree.body[idx]

In [411]:
# we want to find curr's subroutines

f = io.StringIO()
with redirect_stdout(f):
    Return_Call_Nodes().visit(curr)
out = f.getvalue()
out.split()

['c', 'b']

In [412]:
for target in out.split():
    idx = find_idx(target)
    print(idx)

>> found c
def c():
    return 3

3
>> found b
def b():
    return 2 + c()

2


In [133]:
# https://stackoverflow.com/questions/37217823/how-to-find-detect-if-a-build-in-function-is-used-in-python-ast

def foo():
    print('hello')

ast.parse(inspect.getsource(foo))

In [79]:
# generate a list of names of functions that will be called

from collections import deque

class FuncCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()

    @property
    def name(self):
        return '.'.join(self._name)

    @name.deleter
    def name(self):
        self._name.clear()

    def visit_Name(self, node):
        self._name.appendleft(node.id)

    def visit_Attribute(self, node):
        try:
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)


def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FuncCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append(callvisitor.name)

    return func_calls

In [80]:
get_func_calls(tree)

['a', 'c', 'c', 'b']